
# Graphing Puzzle

Write a function answer(dimensions, shooter, target, distance) that gives an array of 2 integers of the width and height of the room, an array of 2 integers of shooter's x and y coordinates in the room, an array of 2 integers of the target's x and y coordinates in the room, and returns an integer of the number of distinct directions that the shooter can fire to hit the target , given the maximum distance that the beam can travel. The beam reflects off of the walls with no loss.

The room has integer dimensions [1 < x_dim <= 1000, 1 < y_dim <= 1000]. The shooter and the target are both positioned on the integer lattice at different distinct positions (x, y) inside the room such that [0 < x < x_dim, 0 < y < y_dim]. Finally, the maximum distance that the beam can travel before becoming harmless will be given as an integer 1 < distance <= 10000.

For example, if the shooter and the target were positioned in a room with dimensions [3, 2], shooter_position [1, 1], target_position [2, 1], and a maximum shot distance of 4, you could shoot in seven different directions to hit the target (given as vector bearings from the shooter location): [1, 0], [1, 2], [1, -2], [3, 2], [3, -2], [-3, 2], and [-3, -2]. As specific examples, the shot at bearing [1, 0] is the straight line horizontal shot of distance 1, the shot at bearing [-3, -2] bounces off the left wall and then the bottom wall before hitting the target with a total shot distance of $\sqrt{13}$ , and the shot at bearing [1, 2] bounces off just the top wall before hitting the target with a total shot distance of $\sqrt{5}$.

Test cases
==========

Inputs:<br>
    * (int list) dimensions = [3, 2]
    * (int list) captain_position = [1, 1]
    * (int list) badguy_position = [2, 1]
    * (int) distance = 4
Output:
    * (int) 7

Inputs:
    * (int list) dimensions = [300, 275]
    * (int list) captain_position = [150, 150]
    * (int list) badguy_position = [185, 100]
    * (int) distance = 500
Output:
    * (int) 9

In [1]:
from math import atan2 as atan2
from math import hypot as hypot


def answer(dimenension, captain_position, bad_guy_position, distance):

    # determine size of grid of mirrored boxes
    hy = float(distance)/dimenension[1]
    hx = float(distance)/dimenension[0]
    grid = (int(max(hy,hx)) + int(max(hy,hx)%1 >0))*2 + 1 # this will be dimensions of grid
    center = grid/2 # find midpoint of grid 

    # make two matrices, distance from me(shooter) to guard (target), distance from me to other
    # me, each tuple is (delta_x, delta_y)
    # make two vectors to fill each matrix
    # one for delta x - same for each column
    # one for detal y - same for each row

    delta_guard = [[(0,0) for _ in range(grid)] for _ in range(grid)]
    dg_x = [0 for _ in range(grid)]
    dg_y = [0 for _ in range(grid)]
    dg_t = [0. for _ in range(grid)]

    delta_me = [[(0,0) for _ in range(grid)] for _ in range(grid)]
    dm_x = [0 for _ in range(grid)]
    dm_y = [0 for _ in range(grid)]
    dm_t = [0. for _ in range(grid)]

    # set the values in four boxes touching the origin of the original box
    # layout matches how boxes would appear on paper (x+ y+ is lower right quatrant of matrix)
    i = center # adjusted for python starting at 0
    dg_x[i] = bad_guy_position[0]-captain_position[0]
    dg_x[i-1] = -bad_guy_position[0]-captain_position[0]
    dg_y[i] = bad_guy_position[1]-captain_position[1]
    dg_y[i-1] = -bad_guy_position[1]-captain_position[1]

    dm_x[i] = 0
    dm_x[i-1] = -2*captain_position[0]
    dm_y[i] = 0
    dm_y[i-1] = -2*captain_position[1]

    # fill positive quadrant (lower right) of matrix
    for j in range(center+1,grid):
        if (j-center)%2 == 0:
            dg_x[j] = (j - center)*dimenension[0] + bad_guy_position[0] - captain_position[0]
            dg_y[j] = (j - center)*dimenension[1] + bad_guy_position[1] - captain_position[1]
            dm_x[j] = (j - center)*dimenension[0]
            dm_y[j] = (j - center)*dimenension[1]
        else:
            dg_x[j] = (j - center + 1)*dimenension[0] - bad_guy_position[0] \
                       - captain_position[0]
            dg_y[j] = (j - center + 1)*dimenension[1] - bad_guy_position[1] \
                       - captain_position[1]
            dm_x[j] = (j - center + 1)*dimenension[0] - 2*captain_position[0]
            dm_y[j] = (j - center + 1)*dimenension[1] - 2*captain_position[1]

    # fill negative quadrant (upper left) of matrix
    for i in range(center - 1):
        if (center-1-i)%2 == 0:
            dg_x[i] = -(center - 1 - i)*dimenension[0] - bad_guy_position[0] \
                      - captain_position[0]
            dg_y[i] = -(center - 1 - i)*dimenension[1] - bad_guy_position[1] \
                      - captain_position[1]
            dm_x[i] = -(center - 1 - i)*dimenension[0] - 2*captain_position[0]
            dm_y[i] = -(center - 1 - i)*dimenension[1] - 2*captain_position[1]
        else:
            dg_x[i] = -(center - 1 - i + 1)*dimenension[0] + bad_guy_position[0] \
                      - captain_position[0]
            dg_y[i] = -(center - 1 - i + 1)*dimenension[1] + bad_guy_position[1] \
                      - captain_position[1]
            dm_x[i] = -(center - 1 - i + 1)*dimenension[0] 
            dm_y[i] = -(center - 1 - i + 1)*dimenension[1] 

    # set the matrix of x,y co-ordiantes relative to original captain (me)
    delta_guard = [[(dg_x[i],dg_y[j]) for i in range(grid)] for j in range(grid)]
    delta_me = [[(dm_x[i],dm_y[j]) for i in range(grid)] for j in range(grid)]

    # then flatten into lists
    bad_guy = [item for sublist in delta_guard for item in sublist]
    good_guy = [item for sublist in delta_me for item in sublist]

    # shots - list of all possible shots captain can take
    # (angle in radians, distance, bg or gg)

    shots = []
    for i in bad_guy:
        r = hypot(i[0],i[1]) 
        if r <= distance:
            shots.append((atan2(i[1],i[0]),r,'bg'))

    for i in good_guy:
        r = hypot(i[0],i[1]) 
        if r <= distance and r != 0:
            shots.append((atan2(i[1],i[0]),r,'gg'))
    shots.sort(key = lambda x: x[0])

    # list of unique theta's in shots (no duplicates)
    thetas = list(set([item[0] for item in shots]))

    # create a dictionary of shots, key = theta
    # set value equal to closes shot at each angle
    # max distance for shot is 10,000
    d_shots = {}
    d_shots = {item: (10010,'empty') for item in thetas}
    for item in shots:
        if item[1] < d_shots[item[0]][0]:
            d_shots[item[0]] = (item[1],item[2])

    hits = 0
    for key,values in d_shots.iteritems():
        if values[1] == 'bg':
            hits += 1
        else:
            continue

    return hits


In [2]:
# Test input 1

box = [3, 2]
me = [1, 1]
bg = [2, 1]
dist = 4

In [4]:
# Test input 2

box = [300, 275]
me = [150, 150]
bg = [185, 100] # bg is bad guy
dist = 500

In [5]:
answer(box,me,bg,dist)


9